In [5]:
import tkinter as tk
from tkinter import messagebox
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas


In [6]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
mapping = {}

def map_letter(letter):
    source = selected.get()
    if source:
        if letter in mapping.values():
            messagebox.showerror("Error bitch" , "You already used that letter")
            update_mapping()  # Ensure colors update
            return
        mapping[source] = letter
        update_mapping()
        selected.set('')

map_buttons = []  # Store mapped letter input row buttons

def update_mapping():
    for btn in buttons:
        letter = btn.letter
        mapped = mapping.get(letter, '_')
        btn.config(text=f"{letter}\n-> {mapped}")
    # Update color for mapped letter buttons
    for btn in map_buttons:
        if btn.letter in mapping.values():
            btn.config(bg='red')
        else:
            btn.config(bg='SystemButtonFace')

root = tk.Tk()
root.title("Subsitution Cipher Mapper")

# Set grid weights for even spacing
for i in range(26):  # For columns (enough for all letters)
    root.grid_columnconfigure(i, weight=1, uniform="col")
for i in range(8):  # For rows (adjust as needed for your layout)
    root.grid_rowconfigure(i, weight=1, uniform="row")

tk.Label(root, text="Enter the text you want encoded (letters only):").grid(row=0, column=0, columnspan=26, pady=5)
input_var = tk.StringVar()
input_entry = tk.Entry(root, textvariable=input_var, width=30)
input_entry.grid(row=1, column=0, columnspan=26, padx=5, pady=5)
value = input_var.get()
selected = tk.StringVar()
buttons = []
selected_btn = [None]  # Track the selected button

def assign_letter(letter):
    selected.set(letter)
    if selected_btn[0]:
        selected_btn[0].config(bg='SystemButtonFace')
    for btn in buttons:
        if btn.letter == letter:
            btn.config(bg='yellow')
            selected_btn[0] = btn
            break

# First row of alphabet buttons (row=2)
for i, letter in enumerate(alphabet):
    btn = tk.Button(root, text=f"{letter}\n->_", width=5, height=3, command=lambda l=letter: assign_letter(l))
    btn.letter = letter
    btn.grid(row=2, column=i, padx=6, pady=6, sticky="nsew")
    buttons.append(btn)

# Second row of alphabet buttons (row=3)
for i, letter in enumerate(alphabet):
    btn = tk.Button(root, text=letter, width=3, command=lambda l=letter: map_letter(l))
    btn.letter = letter
    btn.grid(row=3, column=i, padx=2, sticky="nsew")
    map_buttons.append(btn)

# Centered Selected label/value
frame_selected = tk.Frame(root)
frame_selected.grid(row=4, column=0, columnspan=26, sticky="ew", padx=5, pady=5)

tk.Label(root, text="Encode?").grid(row=4, column=0, columnspan=26)
def show_encoded():
    plain_text = input_var.get().upper()
    encoded = ""
    for char in plain_text:
        if char in mapping:
            encoded += mapping[char]
        else:
            encoded += char
    messagebox.showinfo("Encoded Message", encoded)

tk.Button(root, text="Yes", command=show_encoded).grid(row=4, column=2, columnspan=26, pady=5)
plain_text = input_var.get().upper()
encoded = ""
for char in plain_text:
    if char in mapping:
        encoded += mapping[char]
    else:
        encoded += char
print(encoded)

tk.Button(root, text="Download Key", command=save_key_pdf).grid(row=5, column=0, columnspan=26, pady=5)

def save_key_pdf():
    filename = "substitution_key.pdf"
    c = canvas.Canvas(filename, pagesize=letter)
    c.setFont("Helvetica", 14)
    c.drawString(100, 750, "Substitution Cipher Key")
    y = 720
    for k in sorted(mapping.keys()):
        c.drawString(100, y, f"{k} -> {mapping[k]}")
        y -= 20
        if y < 50:
            c.showPage()
            y = 750
    c.save()
    messagebox.showinfo("PDF Saved", f"Key saved as {filename}")
    
tk.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ts0019\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\ts0019\AppData\Local\Temp\ipykernel_36608\1361830988.py", line 110, in save_key_pdf
    c.save()
    ~~~~~~^^
  File "c:\Users\ts0019\AppData\Local\Programs\Python\Python313\Lib\site-packages\reportlab\pdfgen\canvas.py", line 1300, in save
    if len(self._code): self.showPage()
                        ~~~~~~~~~~~~~^^
  File "c:\Users\ts0019\AppData\Local\Programs\Python\Python313\Lib\site-packages\reportlab\pdfgen\canvas.py", line 648, in showPage
    pageHeight = self._pagesize[1]
                 ~~~~~~~~~~~~~~^^^
IndexError: string index out of range
